In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid,KFold

In [8]:
def feature_importance(data,features,param,inner_cv,outer_cv,Classifier,parameter_grid):
    
    data=data.sample(frac=1)
    X,y=np.array(data[features]), np.array(data[param])

    outer_scores=[]
    df_all= {}
    i=0
    
    for training_samples,test_samples in outer_cv.split(X,y):
        best_parms={}
        best_score=-np.inf
        i+=1
        for parameters in parameter_grid:
            cv_scores=[]
    
            for inner_train,inner_test in inner_cv.split(X[training_samples],y[training_samples]):
                clf=Classifier(**parameters)
                clf.fit(X[inner_train],y[inner_train])
                score=clf.score(X[inner_test],y[inner_test])
                cv_scores.append(score)
                
            mean_score=np.mean(cv_scores)
            
            if mean_score>best_score:
                best_score=mean_score
                best_params=parameters
                
        clf=Classifier(**best_params)
        clf.fit(X[training_samples],y[training_samples])
        outer_scores.append(clf.score(X[test_samples],y[test_samples]))
        
        
        ipdict = {n:s for n, s in zip(features, clf.feature_importances_)} #重要度を変数と対応させて保存
        ser = pd.Series(ipdict) #pd.Seriesに変換
        df_all[i]=ser
        
    results=pd.concat([df_all[key] for key in df_all.keys()],axis=1,sort=False)
    mean=results.mean(axis='columns')
    mean=pd.Series(mean,name="mean")
    outer_scores=pd.Series(outer_scores,name="score")
    table=pd.concat([results,mean],axis=1)
    table=table.sort_values('mean', ascending=False).T
    table=pd.concat([outer_scores,table],axis=1)
    table=table.sort_values('score', ascending=False)
    
    table['score'].values[i]= outer_scores.mean()
    b=[]
    
    for a in range(1,i+1):
        a=str(a)
        b.append(a)
    
    b.append("mean")
    table.index=b
    
    table.to_csv("feature_importance.csv")
    
    return table

In [ ]:
param_grid = {"learning_rate":[i/1000 for i in range(1,501,1)],  
                   "max_depth":[2,3,4,5],
                   "n_estimators":[300]}

In [4]:
data=pd.read_excel("Data.xlsx", sheet_name='regular')#Select regular or time-saving
data

,medium ID,ML round,L-Arginine hydrochloride,L-Glutamine,L-Histidine hydrochloride-H2O,L-Isoleucine,L-Leucine,L-Lysine hydrochloride,L-Methionine,L-Phenylalanine,...,Sodium Chloride (NaCl),Sodium Phosphate monobasic (NaH2PO4-H2O),D-Glucose (Dextrose),Phenol Red,FBS,Penicillin-Streptomycin,cells_ml_at_96h,cells_ml_at_168h,data processing,mode
0,1,0,0.60,2.0,0.2,0.4,0.40,0.4,0.1,0.2,...,120.0,1.0,5.6,0.03,0.1,0.01,77040.000000,529251.428571,"C, FC",initial
1,1,0,0.60,2.0,0.2,0.4,0.40,0.4,0.1,0.2,...,120.0,1.0,5.6,0.03,0.1,0.01,76200.000000,506914.285714,"C, FC",initial
2,2,0,0.60,2.0,0.2,0.4,0.40,0.4,0.1,0.2,...,120.0,1.0,5.6,0.03,0.1,0.01,15171.428571,37800.000000,"C, FC",initial
3,3,0,0.60,2.0,0.2,0.4,0.40,0.4,0.1,0.2,...,120.0,1.0,5.6,0.03,0.1,0.01,96300.000000,612642.857143,"C, FC",initial
4,4,0,0.60,2.0,0.2,0.4,0.40,0.4,0.1,0.2,...,120.0,1.0,5.6,0.03,0.1,0.01,77785.714286,507600.000000,"C, FC",initial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,366,4,0.06,10.0,0.2,0.4,0.40,0.4,1.0,0.2,...,120.0,0.0,5.6,0.03,0.1,0.01,52328.571429,312814.285714,C,R
367,367,4,6.00,5.0,0.2,0.4,0.40,0.4,1.0,0.2,...,120.0,1.0,5.6,0.03,0.1,0.01,46028.571429,280800.000000,C,R
368,368,4,0.60,2.0,0.2,4.0,0.40,0.4,0.1,0.2,...,120.0,0.0,5.6,0.03,0.1,0.01,46800.000000,392914.285714,C,R
369,369,4,6.00,10.0,0.2,4.0,0.04,0.4,1.0,0.2,...,120.0,1.0,5.6,0.03,0.1,0.01,43714.285714,288000.000000,C,R


In [6]:
features=data.columns[2:33]
features

Index(['L-Arginine hydrochloride', 'L-Glutamine',
       'L-Histidine hydrochloride-H2O', 'L-Isoleucine', 'L-Leucine',
       'L-Lysine hydrochloride', 'L-Methionine', 'L-Phenylalanine',
       'L-Threonine', 'L-Tryptophan', 'L-Tyrosine disodium salt dihydrate',
       'L-Valine', 'L-Cystine 2HCl', 'Choline chloride',
       'D-Calcium pantothenate', 'Folic Acid', 'Niacinamide',
       'Pyridoxal hydrochloride', 'Riboflavin', 'Thiamine hydrochloride',
       'i-Inositol', 'Calcium Chloride (CaCl2) (anhyd.)',
       'Magnesium Sulfate (MgSO4) (anhyd.)', 'Potassium Chloride (KCl)',
       'Sodium Bicarbonate (NaHCO3)', 'Sodium Chloride (NaCl)',
       'Sodium Phosphate monobasic (NaH2PO4-H2O)', 'D-Glucose (Dextrose)',
       'Phenol Red', 'FBS', 'Penicillin-Streptomycin'],
      dtype='object')

In [ ]:
feature_importance(data,
                   features,
                   "cells_ml_at_168h",
                   KFold(5,shuffle=True),
                   KFold(5,shuffle=True),
                   GradientBoostingRegressor,
                   ParameterGrid(param_grid))